In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
import os
#import xgboost as xgb
#import lightgbm as lgb
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import gc
warnings.filterwarnings('ignore')


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv('dataset-0510/train.csv')
test  = pd.read_csv('dataset-0510/test.csv')
data = pd.concat([train, test], axis=0)

In [4]:
train['city'].dtypes

dtype('int64')

In [6]:
def feature_process(df):
    #floor
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor']
    
    #location
    df['location_1']  = int(str(df['city']) + (df['town']) + str(df['village']))
    df['location_2']  = int(str(df['city']) + (df['town']))
    
    #date
    df['day_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['year_between_txn_complete'] = df['dt_between_txn_complete'] / 365
    
    #area
    df['land*bulid_area'] = df['land_area'] * df ['building_area']
    df['land/bulid_area'] = df['land_area'] / df ['building_area']
    df['land+bulid_area'] = df['land_area'] + df ['building_area']
    
    #parking
    df['parking_price_every_area'] = df['parking_price'] / df['parking_area']
    
    #population
    
    #rate
    df['positive_born_rate']     = df['born_rate'] / df['death_rate']
    df['positive_marriage_rate'] = df['marriage_rate'] / df['divorce_rate']
    return df

In [31]:
train['parking_way'].value_counts()

2    46065
0    10923
1     3012
Name: parking_way, dtype: int64